In [1]:
import pandas as pd
import json
import re
import warnings
warnings.filterwarnings('ignore')


# استخراج داده ها


<div dir="rtl" style="text-align: right;">
تابع زیر از فایل های json ابتدایی ما فیچر هایی را استخراج میکند
در این پروژه ابتدا فیچر ها انتخاب شده و سپس با روش backward Elimination فیچر های کم اهمیت حذف شده

</div>

In [2]:
def extract_features_from_dataset(df):
    # این تابع بررسی میکند ایا موبایل از اینترنت نسل چهار استفاده میکند یا خیر
    df["4G"] = df["Network"].apply(            
        lambda x: any("4G bands" in item for item in x) if isinstance(x, list) else False   
    )
    # این تابع بررسی میکند ایا موبایل از اینترنت نسل پنج استفاده میکند یا خیر

    df["5G"] = df["Network"].apply(
        lambda x: any("5G bands" in item for item in x) if isinstance(x, list) else False
    )
    # تاریخ معرفی موبایل را استخراج میکند
    df["Announced"] = df["Launch"].apply(
        lambda x: x[0].get("Announced") if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict) else None
    )
    # وضعیت موجودی آن در بازار را استخراج میکند
    df["Status"] = df["Launch"].apply(
        lambda x: x[1].get("Status") if isinstance(x, list) and len(x) > 1 and isinstance(x[1], dict) else None
    )

    #مشخصات فیزیکی محصول

    df["Weight"] = df["Body"].apply(
        lambda x: x[1].get("Weight") if isinstance(x, list) and len(x) > 1 and isinstance(x[1], dict) else None
    )
    df["Length"] = df["Body"].apply(
        lambda x: x[0].get("Dimensions").split(" x ")[0]
        if isinstance(x, list) and len(x) > 0 and "Dimensions" in x[0] and len(x[0]["Dimensions"].split(" x ")) > 0 else None
    )
    df["Width"] = df["Body"].apply(
        lambda x: x[0].get("Dimensions").split(" x ")[1]
        if isinstance(x, list) and len(x) > 0 and "Dimensions" in x[0] and len(x[0]["Dimensions"].split(" x ")) > 1 else None
    )
    df["Diameter"] = df["Body"].apply(
        lambda x: x[0].get("Dimensions").split(" x ")[2].split()[0]
        if isinstance(x, list) and len(x) > 0 and "Dimensions" in x[0] and len(x[0]["Dimensions"].split(" x ")) > 2 else None
    )
    # ایا گوشی ضد آب هست یا خیر
    df["Has IP68"] = df["Body"].apply(
        lambda x: "IP68" in str(x) if isinstance(x, list) else False
    )
    '''
    استخراج باتری با رجکس
    (\d+) عبارت منظمی که به دنبال حداقل یک رشته ای از اعداد میگردد
    \s*   به معنی فضای خالی با بستار ستاره به معنی شروع از صفر
    mAh  که در تمام مشخصات موبایل به این شکل امده است
    .group(1) اولین گروه پرانتز گذاری شده را برمیگرداند

    '''
    df["Battery_capacity"] = df["Battery"].apply(
        lambda x: int(re.search(r"(\d+)\s*mAh", x[0].get("Type")).group(1))
        if isinstance(x, list) and len(x) > 0 and "Type" in x[0] and re.search(r"(\d+)\s*mAh", x[0].get("Type"))
        else None
    )

    #نوع تکنولوژی صفحه نمایش را برمیگرداند 
    df["Display Type"] = df["Display"].apply(
        lambda x: x[0].get("Type").split()[0]
        if isinstance(x, list) and len(x) > 0 and "Type" in x[0] else None
    )
    # سایز نمایشگر
    def extract_display_size(x):
        try:
            if isinstance(x, list) and len(x) > 1 and "Size" in x[1]:
                size_str = x[1]["Size"].split()[0]
                if size_str.replace(".", "", 1).isdigit():
                    return float(size_str)
            return None
        except (IndexError, ValueError):
            return None
    df["Display Size"] = df["Display"].apply(extract_display_size)
    
    # رزولیشن نمایشگر را برمیگرداند
    df["Resolution"] = df["Display"].apply(
        lambda x: x[2].get("Resolution").split(" pixels")[0]
        if isinstance(x, list) and len(x) > 2 and "Resolution" in x[2] and isinstance(x[2]["Resolution"], str) else None
    )
    # فیچری به معنای نسبت صفحه نمایش به کل بدنه به درصد
    df["Body Ratio"] = df["Display"].apply(
        lambda x: x[1].get("Size").split("~")[1].split("%")[0]
        if isinstance(x, list) and len(x) > 1 and "Size" in x[1] and "~" in x[1]["Size"] else None
    )
    # تراکم پیکسلی
    df["PPI"] = df["Display"].apply(
        lambda x: x[2]["Resolution"].split("~")[1].split()[0]
        if isinstance(x, list) and len(x) > 2 and "Resolution" in x[2] and "~" in x[2]["Resolution"]
        else None
    )
    #کیفیت دوربین اصلی
    df["Main Camera Quality"] = df["Main Camera"].apply(
        lambda x: list(x[0].values())[0].split(",")[0]
        if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict) else None
    )
    #کیفیت دوربین سلفی
    df["Selfie Camera Quality"] = df["Selfie camera"].apply(
        lambda x: list(x[0].values())[0].split(",")[0]
        if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict) else None
    )
    # تعداد دوربین یک موبایل
    def extract_camera_count(camera_info):
        try:
            camera_type = list(camera_info[0].keys())[0]
            return {"single": 1, "dual": 2, "triple": 3, "quad": 4}.get(camera_type.lower(), 0)
        except:
            return None
    df["Camera Count"] = df["Main Camera"].apply(
        lambda x: extract_camera_count(x) if isinstance(x, list) and len(x) > 0 else None
    )
    '''
    استخراج بیشترین فرکانس با رجکس
    \d+  به دنبال حداقل یک عدد میگردد
    \. به دنبال نقطه زیرا همه فرکانس ها به این صورت نوشته شده
    \d+  به دنبال دوباره یک عدد بعد از نقطه میگردد
    '''
    
    def extract_max_frequency(cpu_info):
        try:
            frequencies = [float(freq[:-4]) for freq in re.findall(r"\d+\.\d+ GHz", cpu_info)]
            return max(frequencies) if frequencies else None
        except:
            return None
    df["Max Frequency (GHz)"] = df["Platform"].apply(
        lambda x: extract_max_frequency(x[2].get("CPU"))
        if isinstance(x, list) and len(x) > 2 and "CPU" in x[2] else None
    )
    
    """
    استخراج معماری پردازنده با رجکس
    که تا جایی که بررسی شد همراه با واژه CORTEX
    A-Z به دنبال حروف انگلیسی از 
    0-9 به دنبال اعداد از 
    """
    
    
    def extract_architecture(cpu_info):
        try:
            architectures = re.findall(r"Cortex-[A-Z0-9]+", cpu_info)
            return architectures if architectures else None
        except:
            return None
    df["CPU Architecture"] = df["Platform"].apply(
        lambda x: extract_architecture(x[2].get("CPU"))
        if isinstance(x, list) and len(x) > 2 and "CPU" in x[2] else None
    )
    # نام پردازنده
    def extract_chipset_name(chipset_info):
        return chipset_info.split("(")[0].strip() if "(" in chipset_info else chipset_info
    df["Chipset Name"] = df["Platform"].apply(
        lambda x: extract_chipset_name(x[1].get("Chipset"))
        if isinstance(x, list) and len(x) > 1 and "Chipset" in x[1] else None
    )
    # برند تولیدی پردازنده
    def extract_chipset_brand(chipset_info):
        brands = ["Qualcomm", "Apple", "MediaTek", "Exynos"]
        for brand in brands:
            if brand in chipset_info:
                return brand
        return "Other"
    df["Chipset Brand"] = df["Platform"].apply(
        lambda x: extract_chipset_brand(x[1].get("Chipset"))
        if isinstance(x, list) and len(x) > 1 and "Chipset" in x[1] else None
    )
    # پردازنده چند نانومتری است
    def extract_lithography(chipset_info):
        match = re.search(r"\d+ nm", chipset_info)
        return int(match.group(0).split()[0]) if match else None
    df["Lithography (nm)"] = df["Platform"].apply(
        lambda x: extract_lithography(x[1].get("Chipset"))
        if isinstance(x, list) and len(x) > 1 and "Chipset" in x[1] else None
    )
    # بیشترین حافظه پردازنده
    def extract_max_storage(internal_info):
        storage_options = [int(capacity[:-2]) for capacity in re.findall(r'\d+GB', internal_info)]
        return max(storage_options) if storage_options else None
    df["Max Storage (GB)"] = df["Memory"].apply(
        lambda x: extract_max_storage(x[1].get("Internal"))
        if isinstance(x, list) and len(x) > 1 and "Internal" in x[1] else None
    )
    #استخراح بیشترین میزان رم
    def extract_max_ram(internal_info):
        ram_options = [int(capacity.split()[0][:-2]) for capacity in re.findall(r'\d+GB RAM', internal_info)]
        return max(ram_options) if ram_options else None
    df["Max RAM (GB)"] = df["Memory"].apply(
        lambda x: extract_max_ram(x[1].get("Internal"))
        if isinstance(x, list) and len(x) > 1 and "Internal" in x[1] else None
    )

    # ایا موبایل مجهز به بلوتوث است؟
    def extract_bluetooth_version(comms_info):
        for item in comms_info:
            if "Bluetooth" in item:
                version_match = re.search(r'\d+(\.\d+)?', item.get("Bluetooth"))
                return float(version_match.group(0)) if version_match else None
        return None
    df["Bluetooth Version"] = df["Comms"].apply(
        lambda x: extract_bluetooth_version(x) if isinstance(x, list) else None
    )
    # ایا قابلیت پخش صدای بلند دارد
    df["Loudspeaker"] = df["Sound"].apply(
        lambda x: "Yes" in x[0].get("Loudspeaker")
        if isinstance(x, list) and len(x) > 0 and "Loudspeaker" in x[0] else False
    )
    # سیستم عامل موبایل
    df["OS"] = df["Platform"].apply(
        lambda x: x[0].get("OS")
        if isinstance(x, list) and len(x) > 0 and "OS" in x[0] else None
    )
    """
    استخراج قیمت در این داده ها باید به چند صورت انجام شود
    در بعضی از قیمت ها عدد به صورت روپیه هند ثبت شده بود
    و چون موبایل ها از سال های قدیم انتخاب شده بود تا به الان
    با محاسبه تورم هند نسبت به یورو نرخ تبدیل روپیه هند به یورو در حدود 0.125 در نظر گرفته شد
    
    اگر قیمت به صورت یورو ثبت شده باشد هم مستقیم استخراج شده

    """
    def extract_price(misc_info):
        try:
            inr_to_eur = 0.0125

            if isinstance(misc_info, list):
                for item in misc_info:
                    if isinstance(item, dict) and "Price" in item:
                        price_str = item["Price"]

                       
                        if "About" in price_str:
                            if "EUR" in price_str:
                                match = re.search(r"\d+", price_str)
                                return float(match.group()) if match else None
                            elif "INR" in price_str:
                                match = re.search(r"\d+", price_str)
                                return float(match.group()) * inr_to_eur if match else None

                        if "€" in price_str:
                            match = re.search(r"€\s*(\d+\.\d+)", price_str)
                            return float(match.group(1)) if match else None

                        elif "₹" in price_str:
                            match = re.search(r"₹\s*([\d,]+)", price_str)
                            if match:
                                inr_price = float(match.group(1).replace(",", ""))
                                return inr_price * inr_to_eur

                        elif "₹" in price_str and "/" in price_str:
                            match = re.search(r"₹\s*([\d,]+)", price_str)
                            if match:
                                inr_price = float(match.group(1).replace(",", ""))
                                return inr_price * inr_to_eur

            return None
        except Exception as e:
            return None
    df["Price"] = df["Misc"].apply(lambda x: extract_price(x) if isinstance(x, list) else None)


    return df

In [3]:
def clean_body_features(df):
    df["Length"] = df["Length"].apply(
        lambda x: float(x) if isinstance(x, (int, float)) else float(x.split()[0]) if isinstance(x, str) and len(x.split()) > 0 and x.split()[0].isdigit() else None
    )

    df["Width"] = df["Width"].apply(
        lambda x: float(x) if isinstance(x, (int, float)) else float(x.split()[0]) if isinstance(x, str) and len(x.split()) > 0 and x.split()[0].isdigit() else None
    )

    df["Diameter"] = df["Diameter"].apply(
        lambda x: float(x) if isinstance(x, str) and x.replace(".", "").isdigit() else None
    )
    df["Weight"] = df["Weight"].apply(
        lambda x: float(x.split()[0]) if isinstance(x, str) and len(x.split()) > 0 and x.split()[0].replace(".", "").isdigit() else None
    )

    return df

## اعمال تابع بر روی تمامی برند های موبایل 

In [4]:
df = pd.read_json("Sony_data.json")
df.reset_index(inplace=True)
df.rename(columns={"index": "name"}, inplace=True)
df['Sony158'] = df['Sony158'].fillna(df['Sony'])
df.drop(columns=['Sony'], inplace=True)
df_cleaned = pd.json_normalize(df['Sony158'])
df = df_cleaned
df = extract_features_from_dataset(df)
sony = df.drop(columns=['Network', 'Launch', 'Body', 'Display', 'Platform', 'Memory',
                      'Main Camera', 'Selfie camera', 'Sound', 'Comms', 'Features',
                      'Battery', 'Misc', 'Tests', 'Unknown', 'Camera'])
sony2 = clean_body_features(sony)
sony2["brand"] = "sony"

In [5]:
samsung = pd.read_json("Samsung_data.json")

samsung.reset_index(inplace=True)

samsung.rename(columns={"index": "name"}, inplace=True)
sam2 = pd.json_normalize(samsung['Samsung1383'])
sam2 = extract_features_from_dataset(sam2)
sam3 = sam2.drop(columns=['Network', 'Launch', 'Body', 'Display', 'Platform', 'Memory',
                      'Main Camera', 'Selfie camera', 'Sound', 'Comms', 'Features',
                      'Battery', 'Misc', 'Tests', 'Unknown', 'Camera'])
sam4 = clean_body_features(sam3)
sam4["brand"] = "samsung"

In [6]:
alcatel = pd.read_json("Alcatel_data.json")

alcatel.reset_index(inplace=True)

alcatel.rename(columns={"index": "name"}, inplace=True)

alc2 = pd.json_normalize(alcatel['alcatel409'])

alc2 = extract_features_from_dataset(alc2)

alc3 = alc2.drop(columns=['Network', 'Launch', 'Body', 'Display', 'Platform', 'Memory',
                      'Main Camera', 'Selfie camera', 'Sound', 'Comms', 'Features',
                      'Battery', 'Misc', 'Tests', 'Camera'])
alc4 = clean_body_features(alc3)

alc4["brand"] = "Alcatel"

In [7]:
apple = pd.read_json("Apple_data.json")
apple.reset_index(inplace=True)
apple.rename(columns={"index": "name"}, inplace=True)
app2 = pd.json_normalize(apple['Apple118'])
app2 = extract_features_from_dataset(app2)
app3 = app2.drop(columns=['Network', 'Launch', 'Body', 'Display', 'Platform', 'Memory',
                      'Main Camera', 'Selfie camera', 'Sound', 'Comms', 'Features',
                      'Battery', 'Misc', 'Tests','Unknown', 'Camera'])
app4 = clean_body_features(app3)
app4["brand"] = "Apple"

In [8]:
asus = pd.read_json("Asus_data.json")
asus.reset_index(inplace=True)
asus.rename(columns={"index": "name"}, inplace=True)
asu2 = pd.json_normalize(asus['Asus203'])
asu2 = extract_features_from_dataset(asu2)
asu3 = asu2.drop(columns=['Network', 'Launch', 'Body', 'Display', 'Platform', 'Memory',
                      'Main Camera', 'Selfie camera', 'Sound', 'Comms', 'Features',
                      'Battery', 'Misc', 'Tests','Unknown', 'Camera'])
asu4 = clean_body_features(asu3)
asu4["brand"] = "Asus"

In [9]:
blu = pd.read_json("Blu_data.json")
blu.reset_index(inplace=True)
blu.rename(columns={"index": "name"}, inplace=True)
blu2 = pd.json_normalize(blu['BLU369'])
blu2 = extract_features_from_dataset(blu2)
blu3 = blu2.drop(columns=['Network', 'Launch', 'Body', 'Display', 'Platform', 'Memory',
                      'Main Camera', 'Selfie camera', 'Sound', 'Comms', 'Features',
                      'Battery', 'Misc','Camera'])
blu4 = clean_body_features(blu3)
blu4["brand"] = "Blu"

In [10]:
htc = pd.read_json("HTC_data.json")
htc.reset_index(inplace=True)
htc.rename(columns={"index": "name"}, inplace=True)
htc2 = pd.json_normalize(htc['HTC287'])
htc2 = extract_features_from_dataset(htc2)
htc3 = htc2.drop(columns=['Network', 'Launch', 'Body', 'Display', 'Platform', 'Memory',
                      'Main Camera', 'Selfie camera', 'Sound', 'Comms', 'Features',
                      'Battery', 'Misc', 'Tests', 'Unknown'])
htc4 = clean_body_features(htc3)
htc4["brand"] = "Htc"

In [11]:
hua = pd.read_json("Huawei_data.json")
hua.reset_index(inplace=True)
hua.rename(columns={"index": "name"}, inplace=True)
hua2 = pd.json_normalize(hua['Huawei441'])
hua2 = extract_features_from_dataset(hua2)
hua3 = hua2.drop(columns=['Network', 'Launch', 'Body', 'Display', 'Platform', 'Memory',
                      'Main Camera', 'Selfie camera', 'Sound', 'Comms', 'Features',
                      'Battery', 'Misc', 'Tests', 'Unknown', 'Camera'])
hua4 = clean_body_features(hua3)
hua4["brand"] = "Huawei"

In [12]:
inf = pd.read_json("Infinix_data.json")
inf.reset_index(inplace=True)
inf.rename(columns={"index": "name"}, inplace=True)
inf2 = pd.json_normalize(inf['Infinix125'])
inf2 = extract_features_from_dataset(inf2)
inf3 = inf2.drop(columns=['Network', 'Launch', 'Body', 'Display', 'Platform', 'Memory',
                      'Main Camera', 'Selfie camera', 'Sound', 'Comms', 'Features',
                      'Battery', 'Misc', 'Tests', 'Unknown'])
inf4 = clean_body_features(inf3)
inf4["brand"] = "Infinix"

In [13]:
leno = pd.read_json("Lenovo_data.json")
leno.reset_index(inplace=True)
leno.rename(columns={"index": "name"}, inplace=True)
leno2 = pd.json_normalize(leno['Lenovo246'])
leno2 = extract_features_from_dataset(leno2)
leno3 = leno2.drop(columns=['Network', 'Launch', 'Body', 'Display', 'Platform', 'Memory',
                      'Main Camera', 'Selfie camera', 'Sound', 'Comms', 'Features',
                      'Battery', 'Misc', 'Tests', 'Camera'])
leno4 = clean_body_features(leno3)
leno4["brand"] = "Lenovo"

In [14]:
lg = pd.read_json("LG_data.json")
lg.reset_index(inplace=True)
lg.rename(columns={"index": "name"}, inplace=True)
lg2 = pd.json_normalize(lg['LG667'])
lg2 = extract_features_from_dataset(lg2)
lg3 = lg2.drop(columns=['Network', 'Launch', 'Body', 'Display', 'Platform', 'Memory',
                      'Main Camera', 'Selfie camera', 'Sound', 'Comms', 'Features',
                      'Battery', 'Misc', 'Tests','Camera'])
lg4 = clean_body_features(lg3)
lg4["brand"] = "LG"

In [15]:
nok = pd.read_json("Nokia_data.json")
nok.reset_index(inplace=True)
nok.rename(columns={"index": "name"}, inplace=True)
nok2 = pd.json_normalize(nok['Nokia576'])
nok2 = extract_features_from_dataset(nok2)
nok3 = nok2.drop(columns=['Network', 'Launch', 'Body', 'Display', 'Platform', 'Memory',
                      'Main Camera', 'Selfie camera', 'Sound', 'Comms', 'Features',
                      'Battery', 'Misc', 'Tests', 'Unknown','Camera'])
nok4 = clean_body_features(nok3)
nok4["brand"] = "Nokia"

In [16]:
xia = pd.read_json("Xiaomi_data.json")
xia.reset_index(inplace=True)
xia.rename(columns={"index": "name"}, inplace=True)
xia2 = pd.json_normalize(xia['Xiaomi377'])
xia2 = extract_features_from_dataset(xia2)
xia3 = xia2.drop(columns=['Network', 'Launch', 'Body', 'Display', 'Platform', 'Memory',
                      'Main Camera', 'Selfie camera', 'Sound', 'Comms', 'Features',
                      'Battery', 'Misc', 'Tests', 'Unknown','Camera'])
xia4 = clean_body_features(xia3)
xia4["brand"] = "Xiaomi"

In [17]:
zte = pd.read_json("ZTE_data.json")
zte.reset_index(inplace=True)
zte.rename(columns={"index": "name"}, inplace=True)
zte2 = pd.json_normalize(zte['ZTE369'])
zte2 = extract_features_from_dataset(zte2)
zte3 = zte2.drop(columns=['Network', 'Launch', 'Body', 'Display', 'Platform', 'Memory',
                      'Main Camera', 'Selfie camera', 'Sound', 'Comms', 'Features',
                      'Battery', 'Misc', 'Tests', 'Unknown','Camera'])
zte4 = clean_body_features(zte3)
zte4["brand"] = "ZTE"

In [18]:
final_df = pd.concat([sony2, sam4,alc4,app4,asu4,blu4,htc4,hua4,inf4,leno4,lg4,nok4,xia4,zte4], ignore_index=True)

In [19]:
final_df.to_csv('Phone4.csv', index=False)
